<a href="https://colab.research.google.com/github/2022Group3/projectCode/blob/main/basic_model_400epochs_0_85accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from matplotlib import pyplot
# from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
import numpy as np


In [ ]:
# load train and test dataset
def extract_zip():
	# load dataset
  file=np.load(r"drive/MyDrive/data_modified.npz")
  data_npz = dict(zip(("{}".format(k) for k in file), (file[k] for k in file)))
	# one hot encode target values
	# trainY = to_categorical(trainY)
	# testY = to_categorical(testY)
  return data_npz['train'], data_npz['ytrain'], data_npz['validation'], data_npz['yvalidation'], data_npz['test'], data_npz['ytest']

In [ ]:
def load_dataset():
  trainX, trainY, validationX, vlidationY, testX, testY = extract_zip()
  return trainX, to_categorical(trainY), validationX, to_categorical(vlidationY), testX, to_categorical(testY)




In [ ]:
trainX, trainY, validationX, vlidationY, testX, testY = load_dataset()
print(f'trainx= {trainX} , trainy= {trainY}')


trainx= [[[[ 21  57  95]
   [ 21  58  98]
   [ 20  59 101]
   ...
   [ 26  34  46]
   [ 26  35  49]
   [ 29  39  56]]

  [[ 20  55  91]
   [ 20  56  94]
   [ 19  57  97]
   ...
   [ 39  44  53]
   [ 25  33  46]
   [ 22  35  53]]

  [[ 18  52  86]
   [ 18  53  89]
   [ 18  55  93]
   ...
   [ 35  41  53]
   [ 24  34  49]
   [ 24  40  57]]

  ...

  [[231 231 233]
   [242 242 242]
   [230 228 226]
   ...
   [ 27  37  41]
   [ 29  37  37]
   [ 30  38  44]]

  [[181 178 179]
   [209 205 203]
   [192 187 183]
   ...
   [ 23  35  40]
   [ 31  37  37]
   [ 35  37  41]]

  [[126 131 133]
   [102 104 105]
   [ 63  64  63]
   ...
   [ 25  35  46]
   [ 32  37  42]
   [ 35  38  44]]]


 [[[114 110 122]
   [113 109 123]
   [110 105 121]
   ...
   [ 71  72  74]
   [ 37  36  38]
   [ 62  61  63]]

  [[116 112 122]
   [109 105 117]
   [113 109 122]
   ...
   [ 57  60  64]
   [ 43  42  44]
   [ 43  42  44]]

  [[118 115 123]
   [120 116 126]
   [117 112 124]
   ...
   [ 44  48  54]
   [ 47  48  50]
   

In [ ]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm


In [ ]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(16, activation='softmax'))
 	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY,validationX, vlidationY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	# prepare iterator
	it_train = datagen.flow(trainX, trainY, batch_size=64)
	# fit model
	steps = int(trainX.shape[0] / 64)
	history = model.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(validationX, vlidationY), verbose=1)
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=1)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

In [ ]:
# entry point, run the test harness
run_test_harness()

Epoch 1/400
703/703 [==============================] - 37s 34ms/step - loss: 2.6803 - accuracy: 0.2269 - val_loss: 1060.0331 - val_accuracy: 0.0673
Epoch 2/400
703/703 [==============================] - 24s 34ms/step - loss: 2.0718 - accuracy: 0.3281 - val_loss: 924.6054 - val_accuracy: 0.0713
Epoch 3/400
703/703 [==============================] - 26s 37ms/step - loss: 1.9149 - accuracy: 0.3708 - val_loss: 688.1144 - val_accuracy: 0.1168
Epoch 4/400
703/703 [==============================] - 23s 33ms/step - loss: 1.8163 - accuracy: 0.4001 - val_loss: 613.9727 - val_accuracy: 0.1049
Epoch 5/400
703/703 [==============================] - 23s 33ms/step - loss: 1.7538 - accuracy: 0.4201 - val_loss: 634.7299 - val_accuracy: 0.1001
Epoch 6/400
703/703 [==============================] - 27s 38ms/step - loss: 1.6939 - accuracy: 0.4424 - val_loss: 552.9687 - val_accuracy: 0.1069
Epoch 7/400
703/703 [==============================] - 26s 36ms/step - loss: 1.6338 - accuracy: 0.4597 - val_loss: 53